## Inizio Seconda Parte


### Read from bucket

In [1]:
from google.cloud import storage
import pandas as pd
from io import BytesIO
import re
import nltk
from nltk.corpus import stopwords
import string, re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import RobustScaler
%run functions.ipynb

In [2]:
# Your GCS bucket name
bucket_name = 'ccai-storage'

# Paths to the Parquet files in your GCS bucket
articles_parquet_path = 'pipeline/final_df.parquet'

# Initialize the GCS client
client = storage.Client()

# Function to download a Parquet file from GCS and load it into a pandas DataFrame
def load_parquet_from_gcs(bucket_name, file_path):
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_path)
    byte_stream = BytesIO()
    blob.download_to_file(byte_stream)
    byte_stream.seek(0)
    df = pd.read_parquet(byte_stream, engine='pyarrow')
    return df

# Load the Parquet files
final_df0 = load_parquet_from_gcs(bucket_name, articles_parquet_path)

In [3]:
print(final_df0['split'].value_counts())

split
TRAIN       2408
TEST         958
VALIDATE     956
Name: count, dtype: int64


### ----------------------------------------------------------------------------------------

In [4]:
# Carica l'elenco delle stopwords
stop_words = set(stopwords.words('italian'))

# Definisci una funzione per rimuovere le stopwords e la punteggiatura
def preprocess_title(text):

    # Tokenizza il testo
    tokens = nltk.word_tokenize(text)
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rimuovi le stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Join tokens back into a string
    cleaned_text = ' '.join(filtered_tokens)
    
    return cleaned_text

# Applica la funzione remove_stopwords_and_punct alla colonna preprocessed_text del tuo DataFrame
final_df0.loc[:,'body_pre'] = final_df0['body'].apply(preprocess_title)

In [5]:
final_df0

,uri,url,title,body,date,time,dateTime,dateTimePub,lang,isDuplicate,...,alexaGlobalRank,alexaCountryRank,date_column,year,month,year_month,topic,split,shares_scaled,body_pre
0,7582830489,https://www.ilfattoquotidiano.it/2023/06/07/pr...,"Premio Strega 2023, quattro scrittrici in fina...",Quattro scrittrici in cinquina alla finale del...,2023-06-07,19:32:00,2023-06-07T19:32:00Z,2023-06-07T19:29:00Z,ita,False,...,3336,47,2023-06-07,2023,6,2023-6,Arts and Entertainment,TRAIN,0.36,Quattro scrittrici cinquina finale Premio Stre...
1,7358061043,https://www.corriere.it/spettacoli/23_gennaio_...,La rivista Usa The Atlantic stronca i Maneskin...,Il giornalista Spencer Kornhaber fa a pezzi Da...,2023-01-19,18:21:00,2023-01-19T18:21:00Z,2023-01-19T18:10:00Z,ita,False,...,1633,30,2023-01-19,2023,1,2023-1,Arts and Entertainment,TRAIN,6.44,giornalista Spencer Kornhaber fa pezzi Damiano...
2,7376407793,https://www.tgcom24.mediaset.it/people/damiano...,"Damiano David, shopping in gioielleria con Gio...",Per Giorgia Soleri (e Damiano) la gravidanza p...,2023-01-31,09:44:00,2023-01-31T09:44:00Z,2023-01-31T09:43:00Z,ita,False,...,1969,28,2023-01-31,2023,1,2023-1,Arts and Entertainment,TRAIN,0.84,Giorgia Soleri ( Damiano ) gravidanza può aspe...
3,7807122008,https://www.ilfattoquotidiano.it/2023/10/29/x-...,"X Factor, Francesca Michielin e Ambra Angiolin...","Ieri sera, sabato 28 ottobre, Francesca Michie...",2023-10-29,11:03:39,2023-10-29T11:03:39Z,2023-10-29T11:00:59Z,ita,False,...,3336,47,2023-10-29,2023,10,2023-10,Arts and Entertainment,TRAIN,0.36,"Ieri serum , sabato 28 ottobre , Francesca Mic..."
4,7539580387,https://www.corriere.it/spettacoli/23_maggio_1...,Alessandro Cecchi Paone e Simone Antolini hann...,Ancora sconosciute le motivazioni della scelta...,2023-05-11,15:43:00,2023-05-11T15:43:00Z,2023-05-11T15:34:00Z,ita,False,...,1633,30,2023-05-11,2023,5,2023-5,Arts and Entertainment,TRAIN,0.92,Ancora sconosciute motivazioni scelta . celebr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,2023-12-196699085,https://www.repubblica.it/politica/2023/12/19/...,"Azzariti: ""Con il premierato vogliono ridurre ...","Il professore di Diritto costituzionale: ""Le p...",2023-12-19,01:28:03,2023-12-19T01:28:03Z,2023-12-19T00:05:00Z,ita,False,...,1376,8,2023-12-19,2023,12,2023-12,Politics,TEST,0.44,professore Diritto costituzionale : `` parole ...
4318,2023-12-196684238,https://www.repubblica.it/esteri/2023/12/18/ne...,"Gli Usa a Netanyahu: ""Proteggere i civili è un...",Il capo del Pentagono Austin in Israele. Galla...,2023-12-19,01:00:29,2023-12-19T01:00:29Z,2023-12-19T00:00:00Z,ita,False,...,1376,8,2023-12-19,2023,12,2023-12,Politics,TEST,-0.16,capo Pentagono Austin Israele . Gallant apre :...
4319,2023-12-196668947,https://www.repubblica.it/sport/calcio/serie-a...,"La Roma gela Mourinho: altro che rinnovo, prim...",L'apertura del tecnico a restare oltre il 2024...,2023-12-19,00:29:43,2023-12-19T00:29:43Z,2023-12-19T00:00:00Z,ita,False,...,1376,8,2023-12-19,2023,12,2023-12,Sports,TEST,0.28,L'apertura tecnico restare oltre 2024 sbatte F...
4320,2023-12-196655464,https://www.lastampa.it/sport/calcio/2023/12/1...,"Massimo Mauro: ""Un anno senza il mio amico Luc...","""No, vai"".\n\n""Va bene, passo il Natale a casa...",2023-12-19,00:03:20,2023-12-19T00:03:20Z,2023-12-19T00:00:00Z,ita,False,...,5616,91,2023-12-19,2023,12,2023-12,Sports,TEST,-0.16,"`` No , vai '' . `` Va bene , passo Natale cas..."


### ----------------------------------------------------------------------------------------

### Salvo il df come parquet to the bucket

In [7]:
### Salvo il df come parquet to the bucket
bucket_name="ccai-storage"

folder = "pipeline"
# Define your Google Cloud Storage bucket name and Parquet file name
parquet_file_name = 'final_df1.parquet'

# Save DataFrames to GCS as Parquet files in the 'fabio' folder
save_df_to_gcs_parquet(final_df0, bucket_name, folder, parquet_file_name)

#df_no_dups_test
# Remove the local Parquet file
# os.remove(parquet_file_name)

### ----------------------------------------------------------------------------------------

## Fine seconda parte